In [19]:
import os, sys, json
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

import matplotlib.pyplot as plt
from collections import defaultdict

In [20]:
distance_metrics = ["KL-Divergence", "Jensen-Shannon", "Wasserstein-Distance", "Energy-Distance"]

In [21]:
models = {
    "wgan": 'w_gan',
    "wgpgan": 'w_gp_gan',
    "nsgan": 'ns_gan',
    "lsgan": 'ls_gan',
    "mmgan": 'mm_gan',
    "dragan": 'dra_gan',
    "began": 'be_gan',
    "ragan": 'ra_gan',
    "infogan": 'info_gan',
    "fishergan": 'fisher_gan',
    "fgan_forward_kl": 'forkl_gan',
    "fgan_reverse_kl": 'revkl_gan',
    "fgan_jensen_shannon": 'js_gan',
    "fgan_total_var": 'tv_gan',
    "fgan_hellinger": 'hellinger_gan',
    "fgan_pearson": 'pearson_gan',
    "vae": 'vae',
    "autoencoder": 'ae',
}

In [55]:
# def get_best_performance_multivariate(data_type, start_time, trial):
#     # Get path, files in path
#     mypath = "/Users/sob/github/gans6883/hypertuning/{0}/{1}/trial_{2}".format(data_type, start_time, trial)
#     files = [f for f in os.listdir(mypath) if os.path.isfile(os.path.join(mypath, f))]
#     results = []
    
#     # Read in the files
#     for file in files:
#         with open("{}/{}".format(mypath, file)) as f:
#             data = json.load(f)
#         results.append(data)
    
#     # Initialize best dictionary
#     optimal = nested_pickle_dict()
    
#     # Go through all models, distributionss, metrics, and record the best
#     for result in results:
#         for model, distributions in result.items():
#             for distribution, metrics in distributions.items():
#                 for metric, values in metrics.items():
#                     if metric not in ["LR", "HDIM", "BSIZE"]:
                        
#                         # If metric is seen for the first time, it is the best
#                         if metric not in optimal[model][distribution]:
#                             optimal[model][distribution][metric]["value"] = values
#                             optimal[model][distribution][metric]["parameters"] = [metrics["LR"], metrics["HDIM"], metrics["BSIZE"]]
                        
#                         # Otherwise, compare it the presently considered value
#                         elif optimal[model][distribution][metric]["value"][-1] > values[-1]:
#                             optimal[model][distribution][metric]["value"] = values
#                             optimal[model][distribution][metric]["parameters"] = [metrics["LR"], metrics["HDIM"], metrics["BSIZE"]]

#     return optimal

In [65]:
# def get_best_performance_mixture(data_type, start_time, trial):
#     """ For a trial, get the best performance for a mixture model """
#     # Get path, files in path
#     mypath = "/Users/sob/github/gans6883/hypertuning/{0}/{1}/trial_{2}".format(data_type, start_time, trial)
#     files = [f for f in os.listdir(mypath) if os.path.isfile(os.path.join(mypath, f))]
#     results = []
    
#     # Read in the files
#     for file in files:
#         with open("{}/{}".format(mypath, file)) as f:
#             data = json.load(f)
#         results.append(data)
    
#     # Initialize best dictionary
#     optimal = nested_pickle_dict()
#     for result in results:
#         for model, mixtures in result.items():
#             for mixture, distributions in mixtures.items():
#                 for distribution, metrics in distributions.items():
#                     for metric, values in metrics.items():
#                         if metric not in ["LR", "HDIM", "BSIZE"]:
                            
#                             # If metric is seen for the first time, it is the best
#                             if metric not in optimal[model][mixture][distribution]:
#                                 optimal[model][mixture][distribution][metric]["value"] = values
#                                 optimal[model][mixture][distribution][metric]["parameters"] = [metrics["LR"], metrics["HDIM"], metrics["BSIZE"]]

#                             # Otherwise, compare it the presently considered value
#                             elif optimal[model][mixture][distribution][metric]["value"][-1] > values[-1]:
#                                 optimal[model][mixture][distribution][metric]["value"] = values
#                                 optimal[model][mixture][distribution][metric]["parameters"] = [metrics["LR"], metrics["HDIM"], metrics["BSIZE"]]

#     return optimal

In [66]:
# optimal = get_best_performance_multivariate('multivariate', '2018-09-21-1537557860', '1')
# optim = get_best_performance_mixture('mixture', '2018-09-21-1537559182', '1')

In [ ]:
def get_confidence_intervals_multivariate(data_type, start_time):
    """ Compute 95% confidence intervals for multivariate """
    # Files in a file path
    mypath = "/Users/sob/github/gans6883/best/{0}/{1}/".format(data_type, start_time)
    files = [f for f in os.listdir(mypath) if os.path.isfile(os.path.join(mypath, f))]
    results = []
    
    # Load data
    for file in files:
        with open("{}/{}".format(mypath, file)) as f:
            data = json.load(f)
        results.append(data)

    # Initialize optimal
    optimal = nested_pickle_dict()
    for result in results:
        for model, distributions in result.items():
            for dist, metrics in distributions.items():
                for metric, values in metrics.items():
                    if metric not in optimal[model][dist]:
                        optimal[model][dist][metric] = {"original": []}
                    optimal[model][dist][metric]["original"].append(values['value'])

    for result in results:
        for model, distributions in result.items():
            for distribution, metrics in distributions.items():
                for metric, values in metrics.items():
                    data = np.array(optimal[model][distribution][metric]["original"])
                    optimal[model][distribution][metric]['5'] = list(np.percentile(data, 5, axis=0))
                    optimal[model][distribution][metric]['95'] = list(np.percentile(data, 95, axis=0))

    return optimal

In [67]:
mypath = "/Users/sob/github/gans6883/best/{0}/{1}/".format('mixture', '2018-09-21-1537559616')
files = [f for f in os.listdir(mypath) if os.path.isfile(os.path.join(mypath, f))]
results = []

In [69]:
for file in files:
    with open("{}/{}".format(mypath, file)) as f:
        data = json.load(f)
    results.append(data)

In [196]:
optimal = nested_pickle_dict()
for result in results:
    for model, distributions in result.items():
        for dist_i, dists_j in distributions.items():
            for dist_j, metrics in dists_j.items():
                for divergence, output in metrics.items():
                    if divergence not in optimal[model][dist_i][dist_j]:
                        optimal[model][dist_i][dist_j][divergence] = {'values': []}
                    optimal[model][dist_i][dist_j][divergence]['values'].append(output['value'])

In [198]:
optimal[model][dist_i][dist_j]['KL-Divergence']['values']

[[0.34074564603723384, 0.3820046699308716],
 [0.13341187067775861, 0.0895093369321432]]

In [188]:
divergence

'DLoss'

In [199]:
for result in results:
    for model, distributions in result.items():
        for dist_i, dists_j in distributions.items():
            for dist_j, metrics in dists_j.items():
                for divergence, output in metrics.items():
                    data = np.array(optimal[model][mixture][distribution][metric]['values'])
                    optimal[model][mixture][distribution][metric]['5'] = list(np.percentile(data, 5, axis=0))
                    optimal[model][mixture][distribution][metric]['95'] = list(np.percentile(data, 95, axis=0))

In [200]:
optimal[model][mixture][distribution][metric]

{'values': [[0.5270060066431586, 0.25439596801472697],
  [0.1365255235542477, 0.11458112267226735]],
 '5': [0.15604954770869325, 0.12157186493939033],
 '95': [0.507481982488713, 0.24740522574760399]}

In [153]:
output

{'value': [2.0148173570632935, -0.8853402614593506],
 'parameters': [0.2, 16, 128]}

In [141]:
for i, (j, k) in values.items():
    print(i, (j, k))

KL-Divergence ('value', 'parameters')
Jensen-Shannon ('value', 'parameters')
Wasserstein-Distance ('value', 'parameters')
Energy-Distance ('value', 'parameters')
GLoss ('value', 'parameters')
DLoss ('value', 'parameters')


In [120]:
optimal['wgan']['normal']['beta']

defaultdict(<function __main__.nested_pickle_dict()>,
            {'KL-Divergence': [2.0148173570632935, -0.8853402614593506],
             'Jensen-Shannon': [2.0148173570632935, -0.8853402614593506],
             'Wasserstein-Distance': [2.0148173570632935, -0.8853402614593506],
             'Energy-Distance': [2.0148173570632935, -0.8853402614593506],
             'GLoss': [2.0148173570632935, -0.8853402614593506],
             'DLoss': [2.0148173570632935, -0.8853402614593506]})

In [110]:
optimal[model][dist][div]

defaultdict(<function __main__.nested_pickle_dict()>, {})

In [116]:
vals

{'value': [2.0148173570632935, -0.8853402614593506],
 'parameters': [0.2, 16, 128]}

In [112]:
div

'DLoss'

In [146]:
# # def get_confidence_intervals_multivariate(data_type):
# mypath = "/Users/sob/Desktop/mnist_best/"
# files = [f for f in os.listdir(mypath) if os.path.isfile(os.path.join(mypath, f))]
# results = []
# for file in files:
#     print(file)
#     with open("{}/{}".format(mypath, file)) as f:
#         data = json.load(f)
#     results.append(data)
#     data2 = data

optimal = nested_pickle_dict()
for result in results:
    for model, distributions in result.items():
        for dist, metrics in distributions.items():
            for metric, values in metrics.items():
                if metric not in optimal[model][distribution]:
                    optimal[model][dist][metric] = {"original": []}
                optimal[model][dist][metric]["original"].append(values['value'])

# # for result in results:
# #     for model, distributions in result.items():
# #         for distribution, metrics in distributions.items():
# #             for metric, values in metrics.items():
# #                 data = np.array(optimal[gan][distribution][metric]["original"])
# #                 optimal[gan][distribution][metric]['5'] = list(np.percentile(data, 5, axis=0))
# #                 optimal[gan][distribution][metric]['95'] = list(np.percentile(data, 95, axis=0))

# # #     return optimal

results_0_2018-09-10.json
False
results_10_2018-09-15.json
True
results_11_2018-09-16.json
True
results_12_2018-09-16.json
True
results_13_2018-09-16.json
True
results_14_2018-09-16.json
True
results_15_2018-09-17.json
True
results_16_2018-09-17.json
True
results_17_2018-09-17.json
True
results_18_2018-09-18.json
True
results_19_2018-09-18.json
True
results_1_2018-09-11.json
True
results_2_2018-09-11.json
True
results_3_2018-09-12.json
True
results_4_2018-09-12.json
True
results_5_2018-09-13.json
True
results_6_2018-09-13.json
True
results_7_2018-09-13.json
True
results_8_2018-09-14.json
True
results_9_2018-09-14.json
True
